<a href="https://colab.research.google.com/github/netskink/Awesome5/blob/master/ML_collab_testy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 297kB/s 
    100% |████████████████████████████████| 419kB 18.1MB/s 
    100% |████████████████████████████████| 61kB 10.5MB/s 
    100% |████████████████████████████████| 3.0MB 11.2MB/s 


In [7]:
# Load the TensorBoard notebook extension
# In colab this can be done prior to the pip install above.
# I added the pip install since it was done in the dev summit 2019 video
%load_ext tensorboard.notebook 

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# clear any logs from previous runs
!rm -rf ./logs

In [0]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
print(tf.__version__)

2.0.0-alpha0


In [66]:
from google.colab import files
uploaded = files.upload()

Saving usgs_gsvb_v2.csv to usgs_gsvb_v2 (1).csv


In [89]:
!ls

 gdrive   sample_data  'usgs_gsvb_v2 (1).csv'   usgs_gsvb_v2.csv


In [90]:
!head usgs_gsvb_v2.csv

usgs,site_no,date,tidal_high,tidal_low,precipitation_inches,lat,long
GSVB,33670001,2019-03-08,-6.7913385826771648,-6.7257217847769031,,36.735303,-76.011602
GSVB,33670001,2019-03-19,-6.8569553805774275,-6.758530183727034,,36.735303,-76.011602
GSVB,33670001,2019-02-21,1.9356955380577427,2.1325459317585302,,36.735303,-76.011602
GSVB,33670001,2019-03-11,-6.66010498687664,-6.4304461942257216,,36.735303,-76.011602
GSVB,33670001,2019-04-04,1.9028871391076114,2.0669291338582676,,36.735303,-76.011602
GSVB,33670001,2019-02-14,1.7716535433070866,1.9356955380577427,,36.735303,-76.011602
GSVB,33670001,2019-04-14,2.2309711286089238,2.39501312335958,,36.735303,-76.011602
GSVB,33670001,2019-02-24,2.65748031496063,2.9527559055118111,,36.735303,-76.011602
GSVB,33670001,2019-01-12,2.2965879265091864,2.39501312335958,,36.735303,-76.011602


In [166]:
!wc -l usgs_gsvb_v2.csv

9876 usgs_gsvb_v2.csv


In [0]:
num_lines_to_skip = 0

In [0]:
# since some of the fields have missing values, use the optional
# capability of the api
text_lines_ds = tf.data.experimental.CsvDataset(filenames='usgs_gsvb_v2.csv',
                                                record_defaults = [tf.string,                              # Req field: usgs              - STRING  
                                                                   tf.int64,                               # Req field: site_no           - INTEGER  
                                                                   tf.string,                              # Req field: date              - DATE -> STRING  
                                                                   tf.constant([1.2], dtype=tf.float32),   # Opt field: default avg 1.2 Tidal High (feet) - FLOAT
                                                                   tf.constant([-0.89], dtype=tf.float32),   # Opt field: Tidal Low (feet)  - FLOAT
                                                                   tf.constant([0.117], dtype=tf.float32), # Opt field, default annual/365: Precipitation (inches) FLOAT  
                                                                   tf.float32,                             # Req Lat FLOAT
                                                                   tf.float32                              # Long FLOAT
                                                ],  
                                                header=True
                                               ).skip(num_lines_to_skip)

In [161]:
tf.print(text_lines_ds)

<SkipDataset shapes: ((), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>


In [184]:
# Print the initial specified lines in the dataset
# This shows how the missing data is overridden with the defaults for precipitation.
# Also shows how when data is present, eg tidal high it is not overwritten
def my_dataset_head(a_ds, limit=5):
    for i in a_ds:
        limit = limit - 1
        if (limit < 0):
            break
        tf.print(i)
        
my_dataset_head(text_lines_ds, limit=10)

("GSVB", 33670001, "2019-03-08", -6.79133844, -6.72572184, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-19", -6.85695553, -6.75853, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-21", 1.93569553, 2.13254595, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-11", -6.66010475, -6.43044615, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-04", 1.90288711, 2.0669291, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-14", 1.77165353, 1.93569553, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-14", 2.2309711, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-24", 2.65748024, 2.95275593, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-01-12", 2.29658794, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-07", 2.0341208, 2.19816279, 0.117, 36.735302, -76.0116043)


In [191]:
# I don't know how to determine the number or records with TF like pandas.shape
# So iterate through all records and find the number of records.
# Added bonus, it showed that other fields were missing data so the 
# cvsdataset read above needed to be adjusted.
def determine_size_of_dataset(a_ds):
    num_records = 0
    for i in a_ds:
        num_records = num_records + 1
    return(num_records)
 
# BigQuery says that we have 9875 records
DATASET_SIZE = determine_size_of_dataset(text_lines_ds)
print("num records in dataset is {}".format(DATASET_SIZE))

num records in dataset is 9875


In [0]:
# my_ds is a copy of the text_lines_ds so we can manipulate it without rereading the cvs file
my_ds = text_lines_ds

In [0]:
# experiment with shard()
# create two shards and see how the data is split between them.
shard_0_ds = my_ds.shard(2,0)
shard_1_ds = my_ds.shard(2,1)

In [182]:
print("num records in shard 0 dataset is {}".format(determine_size_of_dataset(shard_0_ds)))
print("num records in shard 1 dataset is {}".format(determine_size_of_dataset(shard_1_ds)))

num records in shard 0 dataset is 4938
num records in shard 1 dataset is 4937


In [185]:
# dump the head of shard 0
my_dataset_head(shard_0_ds)

("GSVB", 33670001, "2019-03-08", -6.79133844, -6.72572184, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-21", 1.93569553, 2.13254595, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-04", 1.90288711, 2.0669291, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-14", 2.2309711, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-01-12", 2.29658794, 2.39501309, 0.117, 36.735302, -76.0116043)


In [186]:
# dump the head of shard 1
my_dataset_head(shard_1_ds)

("GSVB", 33670001, "2019-03-19", -6.85695553, -6.75853, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-11", -6.66010475, -6.43044615, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-14", 1.77165353, 1.93569553, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-24", 2.65748024, 2.95275593, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-07", 2.0341208, 2.19816279, 0.117, 36.735302, -76.0116043)


In [0]:
# the dumps shows that the dataset is split by picking every other line since we split on two shards.

In [0]:
# From a stackoverflow post on how to split train, test, and validation 
# datasets using the TF primatives.
# https://stackoverflow.com/questions/48213766/split-a-dataset-created-by-tensorflow-dataset-api-in-to-train-and-test

In [0]:
# Goal is to have three datasets
# Training is 70%
# Validation is 15%
# Testing is 15%

train_size = int(0.7 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)
validation_size = int(0.15 * DATASET_SIZE)

In [195]:
# shuffle the entire dataset
full_ds = my_ds.shuffle(2*DATASET_SIZE)
print("num records in full shuffled dataset is {}".format(determine_size_of_dataset(full_ds)))


num records in full shuffled dataset is 9875


In [196]:
# Good. The dataset did not change in size after shuffle.  Is it reordered?
my_dataset_head(full_ds)

("USGS", 204297575, "2016-04-23", 2.51, -1.83, 0.09, 36.8247795, -75.9852524)
("USGS", 204291317, "2017-09-08", 1.49, -1.22, 0, 36.8433037, -76.1244736)
("USGS", 2042928, "2019-01-06", 0, -0.89, 0.117, 36.8157768, -76.0609131)
("USGS", 2042928, "2018-03-20", 1.17, -0.89, 0.117, 36.8157768, -76.0609131)
("USGS", 204288539, "2018-08-18", 1.39, -1.65, 0.117, 36.7116928, -76.268219)


In [0]:
# It appears so.  
# Lets create the three datasets using take() and skip() as shown in the post.
train_ds = full_ds.take(train_size)  # take from start to train_size for the training set.
test_ds = full_ds.skip(train_size).take(test_size) # skip past the training and take the test_size for test set.
valid_ds = full_ds.skip(train_size).skip(test_size).take(validation_size) # likewise skip twice and take for valid set

In [200]:
# Lets see if sizes are correct
print("num records in full train dataset is {}".format(determine_size_of_dataset(train_ds)))
print("num records in full test dataset is {}".format(determine_size_of_dataset(test_ds)))
print("num records in full valid dataset is {}".format(determine_size_of_dataset(valid_ds)))


num records in full train dataset is 6912
num records in full test dataset is 1481
num records in full valid dataset is 1481


In [0]:
# Lets see if data looks valid on first and last entry of each dataset
def my_dataset_first_and_last(a_ds, size=1):
    count = 0
    for i in a_ds:
        if (count == 0) or (count == size - 1):
            tf.print(i)
        count = count + 1

In [205]:
# dump first and last of train_ds
my_dataset_first_and_last(train_ds, train_size)

("USGS", 204297575, "2016-04-23", 2.51, -1.83, 0.09, 36.8247795, -75.9852524)
("USGS", 204288539, "2018-05-15", 1.83, -2.3, 0.117, 36.7116928, -76.268219)


In [206]:
# dump first and last of test_ds
my_dataset_first_and_last(test_ds, test_size)

("USGS", 204297575, "2016-08-11", 0.79, -1.84, 0, 36.8247795, -75.9852524)
("USGS", 2043155, "2017-01-29", 0.62, -0.66, 0.117, 36.7883606, -76.0707779)


In [207]:
# dump first and last of valid_ds
my_dataset_first_and_last(valid_ds, validation_size)

("USGS", 204300267, "2016-10-04", 0, 1.12, 0.5, 36.6798058, -75.984)
("USGS", 204291317, "2017-09-24", 2.09, -0.41, 0, 36.8433037, -76.1244736)


In [0]:
# that simple spot check looks good, but it worries me that none of these samples
# show a GSVB sample

In [134]:
train_ds

<ShardDataset shapes: ((), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>

In [135]:
# Just print 5 lines including the header
limit = 5
for i in train_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

("GSVB", 33670001, "2019-03-08", -6.79133844, -6.72572184, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-21", 1.93569553, 2.13254595, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-04", 1.90288711, 2.0669291, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-14", 2.2309711, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-01-12", 2.29658794, 2.39501309, 0.117, 36.735302, -76.0116043)


In [0]:
test_ds = my_ds.shard(2,1)

In [137]:
# Just print 5 lines including the header
limit = 5
for i in test_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

("GSVB", 33670001, "2019-03-19", -6.85695553, -6.75853, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-11", -6.66010475, -6.43044615, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-14", 1.77165353, 1.93569553, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-24", 2.65748024, 2.95275593, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-07", 2.0341208, 2.19816279, 0.117, 36.735302, -76.0116043)


In [0]:
d = my_ds.shuffle(shuffle_buffer_size)

TypeError: ignored

In [119]:
# Just print 5 lines including the header
limit = 5
for i in my_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

InvalidArgumentError: ignored

In [0]:
my_ds = text_lines_ds.shard(2,0)
shuffle_buffer_size = 10000
my_ds = my_ds.shuffle(shuffle_buffer_size)
trainset = myds.shard(2,0)
testset = myds.shard(2,1)